## Requirements:
 pip install opencv-contrib-python

## Resize Video

In [1]:
import cv2
import numpy as np

def sharpen_frame(frame):
    # Define a sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1,  5, -1],
                       [0, -1, 0]])
    # Apply the sharpening filter
    sharpened_frame = cv2.filter2D(frame, -1, kernel)
    return sharpened_frame

def process_video(input_file, output_file, width, height):
    # Open video file
    cap = cv2.VideoCapture(input_file)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_file, fourcc, 30.0, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame
        frame_resized = cv2.resize(frame, (width, height))
        # Sharpen the resized frame
        frame_sharpened = sharpen_frame(frame_resized)
        out.write(frame_sharpened)

    cap.release()
    out.release()


In [9]:
# Define paths and dimensions
width, height = 704, 576
process_video(pathResized, pathImprovedPixel, width, height)

## increase the number of frames

In [1]:
pathOriginal = r"LowQualityVideos\Original videos\video1.mp4"
pathResized = r"LowQualityVideos\Resized Videos\video1.mp4"
pathImprovedPixel = r"LowQualityVideos\Repixelated Videos\video1.mp4"
pathImprovedFrame = r"LowQualityVideos\FrameImproved Videos\video1.mp4"

In [62]:
import cv2

def increase_frame_rate(input_video_path, output_video_path, frame_multiplier=2):
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out = cv2.VideoWriter(output_video_path, fourcc, fps * frame_multiplier, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Write the original frame
        out.write(frame)
        # Duplicate frames according to frame_multiplier
        for _ in range(frame_multiplier - 1):
            out.write(frame)

    cap.release()
    out.release()




In [64]:
# Example usage
increase_frame_rate(pathResized, pathImprovedFrame, frame_multiplier=2)

## Track Objects using tracker

In [7]:
import cv2
from tracker import *
from IPython.display import clear_output

In [146]:
# BackgroundThresholdSensitivity
# A lower value (e.g., 20) makes the algorithm more sensitive to small changes, meaning it will detect more subtle movements. Conversely, a higher value makes it less sensitive, requiring larger changes to detect motion.

# ObjectSizeSensitivity:
# ObjectSizeSensitivity is used to filter out contours that are too small to be considered significant objects. This helps in reducing the number of false positives and noise in the object detection process.
# if the number of pixels in a group was less than this number it will not consider it as an object
# Biger number, less objects
detections = []
def ObjectTracker(Path, BackgroundThresholdHistory = 1000, BackgroundThresholdSensitivity  = 20, ObjectSizeSensitivity = 1000, RemoveShadow = True, PlaySpeed = 10):

    countofAllObjectsInBusiestTime = 0
    countofAllobjectInAllVideo = 0
    
    tracker = EuclideanDistTracker()
    cap = cv2.VideoCapture(Path)
    
    object_detector = cv2.createBackgroundSubtractorMOG2(history=BackgroundThresholdHistory, varThreshold = BackgroundThresholdSensitivity)
    # history:
    # This parameter defines the length of the history or the number of previous frames that the algorithm uses to model the background.
    # A higher value means the model will consider more frames, which can be useful in environments where the background changes slowly.
    # In this case, history=100 means the model will use the last 100 frames to learn and update the background model.
    #
    # varThreshold:
    # This is the variance threshold for the pixel-wise comparison.
    # It determines the sensitivity of the algorithm to changes in the pixel values.
    # A lower value (e.g., 20) makes the algorithm more sensitive to small changes, meaning it will detect more subtle movements. Conversely, a higher value makes it less sensitive, requiring larger changes to detect motion.
    
    
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Morphological kernel
    
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break 

            
        height, width, _ = frame.shape
        roi = frame[30:height-15, 0:width] # you can define the area that you are intrested to

        # 1. Object Detection    
        mask = object_detector.apply(roi)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  

        #if RemoveShadow:
            #_, mask = cv2.threshold(mask, 254,254, cv2.THRESH_BINARY) # Removing Gray colors or shadows from mask
        #else:
            #_, mask = cv2.threshold(mask, 0,254, cv2.THRESH_BINARY) # changing Gray colors or shadows from mask to white
            

        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detections = []
        for cnt in contours:
    
            # Calculate area and remove small elements
            area = cv2.contourArea(cnt)

            
            # ObjectSizeSensitivity:
            # ObjectSizeSensitivity is used to filter out contours that are too small to be considered significant objects. This helps in reducing the number of false positives and noise in the object detection process.
            # if the number of pixels in a group was less than this number it will not consider it as an object
            if area > ObjectSizeSensitivity:
                
                # Drowing the object boundary
                #cv2.drawContours(frame, [cnt], -1, (0,0,255), 2)
                x, y, w, h = cv2.boundingRect(cnt)
                
                #Drowing a box for object
                cv2.rectangle(roi, (x,y),(x + w, y + h),(0 ,255, 0),3)
                detections.append([x,y,w,h])
    
        # 2. Object Tracking
        boxes_ids = tracker.update(detections)
        
        for box_id in boxes_ids:
            x, y, w, h, id  = box_id
            #cv2.putText(frame, str(id) + ':' + str(box_id), (x,y - 15), cv2.FONT_HERSHEY_PLAIN, 1, (255,0,0),2) # showing the box cordinate
            cv2.putText(roi, str(id) , (x,y - 15), cv2.FONT_HERSHEY_PLAIN, 1, (255,0,0),2)            
            cv2.rectangle(roi, (x,y),(x + w, y + h),(0 ,255, 0),3)            

            if countofAllobjectInAllVideo < id:
                countofAllobjectInAllVideo = id
            
        if countofAllObjectsInBusiestTime < len(boxes_ids):
            countofAllObjectsInBusiestTime = len(boxes_ids)
        

        clear_output(wait=True)
        
        cv2.imshow("Frame", frame)
        #cv2.imshow("roi", roi)
        cv2.imshow("Mask", mask)

        
        # PlaySpeed is the speed of play
        if cv2.waitKey(PlaySpeed) & 0xFF == 27:
            break
            

    # closing video windoes
    cap.release()
    #cv2.destroyAllWindows()

    return countofAllObjectsInBusiestTime , countofAllobjectInAllVideo 

In [148]:
PeopleWalking = r"Videos\CarinHighway1.mp4"
pathOriginal = r"LowQualityVideos\Original videos\video1.mp4"
pathResized = r"LowQualityVideos\Resized Videos\video1.mp4"
pathImprovedPixel = r"LowQualityVideos\Repixelated Videos\video1.mp4"

In [150]:
result = ObjectTracker(pathOriginal, BackgroundThresholdHistory = 1000, BackgroundThresholdSensitivity = 100, ObjectSizeSensitivity = 100, RemoveShadow = True, PlaySpeed = 30)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])


KeyboardInterrupt: 

## Object Tracker Using sklearn Cluster

In [473]:
#pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [67]:
import cv2
from tracker import *
from IPython.display import clear_output
import numpy as np
from sklearn.cluster import DBSCAN

def ObjectTrackerUsingSklearnCluster(Path, BackgroundThresholdHistory = 1000, BackgroundThresholdSensitivity=20, ObjectSizeSensitivity=1000, RemoveShadow=True, clustering_eps=20, min_samples=2, PlaySpeed=10):

    countofAllObjectsInBusiestTime = 0
    countofAllobjectInAllVideo = 0
    
    tracker = EuclideanDistTracker()  # Initialize your tracker
    cap = cv2.VideoCapture(Path)
    
    object_detector = cv2.createBackgroundSubtractorMOG2(history=BackgroundThresholdHistory, varThreshold=BackgroundThresholdSensitivity)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Morphological kernel
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, _ = frame.shape
        roi = frame[30:height-15, 0:width] # you can define the area that you are intrested to
        
        mask = object_detector.apply(roi)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        if RemoveShadow:
            _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
        else:
            _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
        
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detections = []
        all_points = []
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > ObjectSizeSensitivity:
                x, y, w, h = cv2.boundingRect(cnt)
                detections.append([x, y, w, h])
                
                # Add all contour points to a list for clustering
                for point in cnt:
                    all_points.append(point[0])  # Flatten the point structure

        # Cluster points based on proximity
        if len(all_points) > 0:
            clustering = DBSCAN(eps=clustering_eps, min_samples=min_samples).fit(all_points)
            labels = clustering.labels_
            
            unique_labels = set(labels)
            for label in unique_labels:
                if label == -1:
                    continue
                
                label_points = np.array([all_points[i] for i in range(len(all_points)) if labels[i] == label])
                
                x, y, w, h = cv2.boundingRect(label_points)
                cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)
                detections.append([x, y, w, h])
        
        boxes_ids = tracker.update(detections)
        
        for box_id in boxes_ids:
            x, y, w, h, id = box_id
            cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)            
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)            

            if countofAllobjectInAllVideo < id:
                countofAllobjectInAllVideo = id
            
        if countofAllObjectsInBusiestTime < len(boxes_ids):
            countofAllObjectsInBusiestTime = len(boxes_ids)
        
        clear_output(wait=True)
        
        #cv2.imshow("roi", roi)
        cv2.imshow("Frame", frame)
        cv2.imshow("Mask", mask)
    
        if cv2.waitKey(PlaySpeed) & 0xFF == 27:
            break

    cap.release()
    #cv2.destroyAllWindows()

    return countofAllObjectsInBusiestTime, countofAllobjectInAllVideo



In [69]:
result = ObjectTrackerUsingSklearnCluster(pathOriginal, BackgroundThresholdHistory = 1000, BackgroundThresholdSensitivity = 100, ObjectSizeSensitivity = 100, RemoveShadow = True, clustering_eps=50, min_samples=50, PlaySpeed = 50)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])


Count of object in the busiest frame :  4
Count of All detected object :  18


## Object Tracker Using Self Cluster

In [25]:
import cv2
import numpy as np
from tracker import *
from IPython.display import clear_output


In [71]:

def cluster_contours(detections, distance_threshold=200):
    clusters = []
    for rect in detections:
        x, y, w, h = rect
        added_to_cluster = False
        for cluster in clusters:
            for cx, cy, cw, ch in cluster:
                if abs(x - cx) < distance_threshold and abs(y - cy) < distance_threshold:
                    cluster.append(rect)
                    added_to_cluster = True
                    break
            if added_to_cluster:
                break
        if not added_to_cluster:
            clusters.append([rect])
    return clusters

def ObjectTrackerUsingSelfCluster(Path, BackgroundThresholdHistory = 1000, BackgroundThresholdSensitivity=20, ObjectSizeSensitivity=1000, ObjectsDistanceTreshhold = 200, RemoveShadow=True, PlaySpeed=10):

    countofAllObjectsInBusiestTime = 0
    countofAllobjectInAllVideo = 0
    
    tracker = EuclideanDistTracker()
    cap = cv2.VideoCapture(Path)
    
    object_detector = cv2.createBackgroundSubtractorMOG2(history=BackgroundThresholdHistory, varThreshold=BackgroundThresholdSensitivity)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Morphological kernel
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, _ = frame.shape
        roi = frame[30:height-15, 0:width] # you can define the area that you are intrested to

        
        mask = object_detector.apply(roi)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        if RemoveShadow:
            _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
        else:
            _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
        
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detections = []
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > ObjectSizeSensitivity:
                x, y, w, h = cv2.boundingRect(cnt)
                detections.append([x, y, w, h])

        clusters = cluster_contours(detections,ObjectsDistanceTreshhold)
        
        combined_detections = []
        for cluster in clusters:
            if len(cluster) > 1:
                cluster_x = min([x for x, y, w, h in cluster])
                cluster_y = min([y for x, y, w, h in cluster])
                cluster_w = max([x + w for x, y, w, h in cluster]) - cluster_x
                cluster_h = max([y + h for x, y, w, h in cluster]) - cluster_y
                combined_detections.append([cluster_x, cluster_y, cluster_w, cluster_h])
            else:
                combined_detections.append(cluster[0])
        
        boxes_ids = tracker.update(combined_detections)
        
        for box_id in boxes_ids:
            x, y, w, h, id  = box_id
            cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)            
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

            if countofAllobjectInAllVideo < id:
                countofAllobjectInAllVideo = id
            
        if countofAllObjectsInBusiestTime < len(boxes_ids):
            countofAllObjectsInBusiestTime = len(boxes_ids)
        
        
        clear_output(wait=True)

        #cv2.imshow("roi", roi)
        cv2.imshow("Frame", frame)
        cv2.imshow("Mask", mask)
    
        if cv2.waitKey(PlaySpeed) & 0xFF == 27:
            break

    cap.release()
    #cv2.destroyAllWindows()  

    return countofAllObjectsInBusiestTime, countofAllobjectInAllVideo

# Example usage




In [73]:
result = ObjectTrackerUsingSelfCluster(pathOriginal, BackgroundThresholdHistory = 1000, BackgroundThresholdSensitivity = 100, ObjectSizeSensitivity = 100,ObjectsDistanceTreshhold = 5, RemoveShadow = False, PlaySpeed = 30)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])


Count of object in the busiest frame :  6
Count of All detected object :  30


# Executing Tests

## Low Quality Videos

In [75]:
pathOriginal = r"LowQualityVideos\Original videos\video3.mp4"
# pathResized = r"LowQualityVideos\Resized Videos\video1.mp4"
# pathImprovedPixel = r"LowQualityVideos\Repixelated Videos\video1.mp4"
# pathImprovedFrame = r"LowQualityVideos\FrameImproved Videos\video1.mp4"

In [85]:
result = ObjectTracker(pathOriginal, BackgroundThresholdHistory = 50, BackgroundThresholdSensitivity = 100, ObjectSizeSensitivity = 100, RemoveShadow = True, PlaySpeed = 30)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])

Count of object in the busiest frame :  3
Count of All detected object :  7


In [87]:
result = ObjectTrackerUsingSklearnCluster(pathOriginal, BackgroundThresholdHistory = 100, BackgroundThresholdSensitivity = 100, ObjectSizeSensitivity = 100, RemoveShadow = True, clustering_eps=50, min_samples=50, PlaySpeed = 50)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])

Count of object in the busiest frame :  5
Count of All detected object :  41


In [95]:
result = ObjectTrackerUsingSelfCluster(pathOriginal, BackgroundThresholdHistory = 100, BackgroundThresholdSensitivity = 100, ObjectSizeSensitivity = 100,ObjectsDistanceTreshhold = 10, RemoveShadow = False, PlaySpeed = 30)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])

Count of object in the busiest frame :  9
Count of All detected object :  91


## NormalVideos

In [158]:
pathOriginal = r"Videos\CarinHighway1.mp4"

In [160]:
result = ObjectTracker(pathOriginal, BackgroundThresholdHistory = 100, BackgroundThresholdSensitivity = 500, ObjectSizeSensitivity = 10000, RemoveShadow = True, PlaySpeed = 30)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])

Count of object in the busiest frame :  2
Count of All detected object :  34


In [162]:
result = ObjectTrackerUsingSklearnCluster(pathOriginal, BackgroundThresholdHistory = 100, BackgroundThresholdSensitivity = 1500, ObjectSizeSensitivity = 500, clustering_eps=500, min_samples=1000, PlaySpeed = 50)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])

Count of object in the busiest frame :  7
Count of All detected object :  94


In [140]:
result = ObjectTrackerUsingSelfCluster(pathOriginal, BackgroundThresholdHistory = 100, BackgroundThresholdSensitivity = 1500, ObjectSizeSensitivity = 500,ObjectsDistanceTreshhold = 50, RemoveShadow = False, PlaySpeed = 30)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])

Count of object in the busiest frame :  7
Count of All detected object :  90


In [168]:
result = ObjectTracker(pathOriginal, BackgroundThresholdHistory = 1000, BackgroundThresholdSensitivity = 100, ObjectSizeSensitivity = 10000, RemoveShadow = True, PlaySpeed = 30)
print( "Count of object in the busiest frame : " , result[0])
print( "Count of All detected object : " , result[1])

Count of object in the busiest frame :  3
Count of All detected object :  91
